In [7]:
import json
import random
import pickle
import jsonlines
import tqdm
import networkx as nx

In [2]:
# load dict from file
with open("/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/title2id_dict.pkl", "rb") as f:
    title2id_dict = pickle.load(f)
with open("/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/author2id_dict.pkl", "rb") as f:
    author2id_dict = pickle.load(f)
with open("/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/id2title_dict.pkl", "rb") as f:
    id2title_dict = pickle.load(f)
with open("/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/id2author_dict.pkl", "rb") as f:
    id2author_dict = pickle.load(f)

In [3]:
# Load the graphs from a file
with open('/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/paper_net.pkl', 'rb') as f:
    paper_net = pickle.load(f)

with open('/<YOUR_OWN_PATH>/ToolQA/data/external_corpus/dblp/author_net.pkl', 'rb') as f:
    author_net = pickle.load(f)

In [28]:
num_question_per_template = 10
question_id = 0
questions = []

In [29]:
# How many people does {author1} need to know at least to know {author2} in the DBLP citation network?
for i in tqdm.tqdm(range(num_question_per_template)):
    # randomly choose a paper
    author_id1 = random.choice(list(id2author_dict.keys()))
    name1 = id2author_dict[author_id1]
    candidate_list = list(author_net.neighbors(author_id1))
    while candidate_list == []:
        author_id1 = random.choice(list(id2author_dict.keys()))
        name1 = id2author_dict[author_id1]
        candidate_list = list(author_net.neighbors(author_id1))
    author_id2 = random.choice(candidate_list)
    name2 = id2author_dict[author_id2]
    question = "How many people does {} need to know at least to know {} in the DBLP citation network?".format(name1, name2)
    answer = str(nx.shortest_path_length(author_net, author_id1, author_id2))
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-10:])

100%|██████████| 10/10 [00:00<00:00, 173.47it/s]

[{'qid': 'hard-dblp-0000', 'question': 'How many people does Lili He need to know at least to know Zhikang Shuai in the DBLP citation network?', 'answer': '1'}, {'qid': 'hard-dblp-0001', 'question': 'How many people does Hiroshi Ishiguro need to know at least to know Manabu Ikeda in the DBLP citation network?', 'answer': '1'}, {'qid': 'hard-dblp-0002', 'question': 'How many people does Badawy Reham need to know at least to know Bloem Bastiaan in the DBLP citation network?', 'answer': '1'}, {'qid': 'hard-dblp-0003', 'question': 'How many people does Cagatay Goncu need to know at least to know Samuel Reinders in the DBLP citation network?', 'answer': '1'}, {'qid': 'hard-dblp-0004', 'question': 'How many people does Mark de Reuver need to know at least to know Jafar Rezaei in the DBLP citation network?', 'answer': '1'}, {'qid': 'hard-dblp-0005', 'question': 'How many people does Pratheepan Yogarajah need to know at least to know Sonya Coleman in the DBLP citation network?', 'answer': '1'}

In [30]:
# How many common collaborators does {author} have with {author}?
for i in tqdm.tqdm(range(num_question_per_template)):
    author_id1 = random.choice(list(id2author_dict.keys()))
    name1 = id2author_dict[author_id1]
    candidate_list = list(author_net.neighbors(author_id1))
    while candidate_list == []:
        author_id1 = random.choice(list(id2author_dict.keys()))
        name1 = id2author_dict[author_id1]
        candidate_list = list(author_net.neighbors(author_id1))
    author_id2 = random.choice(candidate_list)
    name2 = id2author_dict[author_id2]
    question = "How many common collaborators does {} have with {}?".format(name1, name2)
    answer = str(len(list(nx.common_neighbors(author_net, author_id1, author_id2))))
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-10:])

100%|██████████| 10/10 [00:00<00:00, 174.10it/s]

[{'qid': 'hard-dblp-0010', 'question': 'How many common collaborators does Luigi Asprino have with Aldo Gangemi?', 'answer': '0'}, {'qid': 'hard-dblp-0011', 'question': 'How many common collaborators does Ananya Misra have with Thibault Doutre?', 'answer': '8'}, {'qid': 'hard-dblp-0012', 'question': 'How many common collaborators does Guoxian Yu have with Qiaoyu Tan?', 'answer': '3'}, {'qid': 'hard-dblp-0013', 'question': 'How many common collaborators does Jinhao Wu have with Jia, Riheng?', 'answer': '4'}, {'qid': 'hard-dblp-0014', 'question': 'How many common collaborators does Abdullah Mohammed Al-Majid have with Sammer Yousuf?', 'answer': '4'}, {'qid': 'hard-dblp-0015', 'question': 'How many common collaborators does Deli Zhao have with Jiapeng Zhu?', 'answer': '4'}, {'qid': 'hard-dblp-0016', 'question': 'How many common collaborators does Zhiwei Yu have with Shuang Chen?', 'answer': '4'}, {'qid': 'hard-dblp-0017', 'question': 'How many common collaborators does Ran Liu have with X

In [31]:
# Which is the most cited paper written by {author} in the DBLP citation network?
for i in tqdm.tqdm(range(num_question_per_template)):
    answer = ""
    while answer == "":
        author_id = random.choice(list(id2author_dict.keys()))
        name = id2author_dict[author_id]
        question = "Which is the most cited paper written by {} in the DBLP citation network?".format(name)
        max_citation = 0
        for neighbour in author_net.neighbors(author_id):
            for paper in author_net[author_id][neighbour]["papers"]:
                if paper_net.nodes[paper]["n_citation"] > max_citation:
                    max_citation = paper_net.nodes[paper]["n_citation"]
                    answer = paper_net.nodes[paper]["title"]
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-10:])

100%|██████████| 10/10 [00:00<00:00, 119.96it/s]

[{'qid': 'hard-dblp-0020', 'question': 'Which is the most cited paper written by Andreas Vlachos in the DBLP citation network?', 'answer': 'FEVEROUS: Fact Extraction and VERification Over Unstructured and Structured information.'}, {'qid': 'hard-dblp-0021', 'question': 'Which is the most cited paper written by Yanshuai Cao in the DBLP citation network?', 'answer': 'Turing: An Accurate And Interpretable Multi-Hypothesis Cross-Domain Natural Language Database Interface'}, {'qid': 'hard-dblp-0022', 'question': 'Which is the most cited paper written by Taru Jain in the DBLP citation network?', 'answer': 'Multitask Learning for Emotionally Analyzing Sexual Abuse Disclosures'}, {'qid': 'hard-dblp-0023', 'question': 'Which is the most cited paper written by Guanyu Lin in the DBLP citation network?', 'answer': 'Dual Contrastive Network for Sequential Recommendation'}, {'qid': 'hard-dblp-0024', 'question': 'Which is the most cited paper written by Yashoteja Prabhu in the DBLP citation network?'

In [32]:
# Which collaborator does {author} have the most cited paper with in the DBLP citation network?
for i in range(num_question_per_template):
    answer = ""
    while answer == "":
        author_id = random.choice(list(id2author_dict.keys()))
        name = id2author_dict[author_id]
        question = "Which collaborator does {} have the most citations with in the DBLP citation network?".format(name)
        
        max_citation = 0
        for neighbour in author_net.neighbors(author_id):
            citation = 0
            for paper in author_net[author_id][neighbour]["papers"]:
                citation += paper_net.nodes[paper]["n_citation"]
            if citation > max_citation:
                max_citation = citation
                answer = id2author_dict[neighbour]
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-1])

{'qid': 'hard-dblp-0039', 'question': 'Which collaborator does S. Thomas George have the most citations with in the DBLP citation network?', 'answer': 'Sanjukta Rani Jena'}


In [33]:
# Which venue does {author} publish the most papers in the DBLP citation network?
for i in range(num_question_per_template):
    answer = ""
    while answer == "":
        author_id = random.choice(list(id2author_dict.keys()))
        name = id2author_dict[author_id]
        question = "Which venue does {} publish the most papers in the DBLP citation network?".format(name)
        
        max_paper = 0
        for neighbour in author_net.neighbors(author_id):
            paper_num = 0
            for paper in author_net[author_id][neighbour]["papers"]:
                paper_num += 1
            if paper_num > max_paper:
                max_paper = paper_num
                answer = [paper_net.nodes[paper]["venue"]["raw"]]
            else:
                answer.append(paper_net.nodes[paper]["venue"]["raw"])
        answer = ",".join(answer)
        questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
        question_id += 1
print(questions[-1])

{'qid': 'hard-dblp-0049', 'question': 'Which venue does Chenguang He publish the most papers in the DBLP citation network?', 'answer': 'IWCMC 2021: 2021 17TH INTERNATIONAL WIRELESS COMMUNICATIONS & MOBILE COMPUTING CONFERENCE (IWCMC),IWCMC 2021: 2021 17TH INTERNATIONAL WIRELESS COMMUNICATIONS & MOBILE COMPUTING CONFERENCE (IWCMC),Mobile Networks and Applications'}


In [34]:
# How many accumulated citations do papers collaborated by {author1} and {author2} have in the DBLP citation network?
for i in range(num_question_per_template):
    author_id1 = random.choice(list(id2author_dict.keys()))
    name1 = id2author_dict[author_id1]
    candidate_list = list(author_net.neighbors(author_id1))
    while candidate_list == []:
        author_id1 = random.choice(list(id2author_dict.keys()))
        name1 = id2author_dict[author_id1]
        candidate_list = list(author_net.neighbors(author_id1))
    author_id2 = random.choice(candidate_list)
    name2 = id2author_dict[author_id2]
    question = "How many accumulated citations do papers collaborated by {} and {} have in the DBLP citation network?".format(name1, name2)
    answer = 0
    for paper in author_net[author_id1][author_id2]["papers"]:
        answer += paper_net.nodes[paper]["n_citation"]
    answer = str(answer)
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-1])

{'qid': 'hard-dblp-0059', 'question': 'How many accumulated citations do papers collaborated by Chathura Wanigasekara and Akshya Swain have in the DBLP citation network?', 'answer': '1'}


In [35]:
# How many papers in all do {author} and his/her collaborators have in the DBLP citation network?
for i in range(num_question_per_template):
    answer = 720000
    while answer > 500000:
        author_id = random.choice(list(id2author_dict.keys()))
        name = id2author_dict[author_id]
        question = "How many papers in all do {} and his/her collaborators have in the DBLP citation network?".format(name)
        answer = []
        answer.append(author_id)
        for neighbour in author_net.neighbors(author_id):
            # answer += len(author_net[author_id][neighbour]["papers"])
            answer.append(neighbour)
            for neighbour2 in author_net.neighbors(neighbour):
                if neighbour2 != author_id:
                    # answer += len(author_net[neighbour][neighbour2]["papers"])
                    answer.append(neighbour2)
        answer = set(answer)
        subgraph = author_net.subgraph(answer)
        answer = subgraph.number_of_edges()
    print(question, answer)
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-1])

How many papers in all do Han Wu and his/her collaborators have in the DBLP citation network? 100
How many papers in all do Guang Cheng and his/her collaborators have in the DBLP citation network? 33
How many papers in all do Jiayu Dong and his/her collaborators have in the DBLP citation network? 10
How many papers in all do Duc-Dung Tran and his/her collaborators have in the DBLP citation network? 593
How many papers in all do Atsushi Keyaki and his/her collaborators have in the DBLP citation network? 63
How many papers in all do Pietro Ferraro and his/her collaborators have in the DBLP citation network? 0
How many papers in all do Chejian Xu and his/her collaborators have in the DBLP citation network? 2133
How many papers in all do Irvin Aloise and his/her collaborators have in the DBLP citation network? 348
How many papers in all do Hongping Wang and his/her collaborators have in the DBLP citation network? 103
How many papers in all do Claire O'Shea and his/her collaborators have in

In [36]:
# Who collarborated with {author} most in the DBLP citation network?
for i in tqdm.tqdm(range(num_question_per_template)):
    # randomly choose a paper
    author_id = random.choice(list(id2author_dict.keys()))
    name = id2author_dict[author_id]
    question = "Who collaborated with {} most in the DBLP citation network?".format(name)
    max_weight = 0
    max_author = []
    for neighbor in author_net.neighbors(author_id):
        if author_net[author_id][neighbor]["weight"] > max_weight:
            max_weight = author_net[author_id][neighbor]["weight"]
            max_author = [id2author_dict[neighbor]]
        elif author_net[author_id][neighbor]["weight"] == max_weight:
            max_author.append(id2author_dict[neighbor])
    answer = ', '.join(max_author)
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-1])

100%|██████████| 10/10 [00:00<00:00, 179.76it/s]

{'qid': 'hard-dblp-0079', 'question': 'Who collaborated with Vincent Lepetit most in the DBLP citation network?', 'answer': 'Sinisa Stekovic, Friedrich Fraundorfer, Mahdi Rad'}


In [37]:
# What institutions participated in the study of {title} in the DBLP citation network?
for i in range(num_question_per_template):
    # randomly choose a paper
    paper_id = random.choice(list(id2title_dict.keys()))
    title = id2title_dict[paper_id]
    question = "What institutions participated in the study of {} in the DBLP citation network?".format(title)
    answer = []
    authors = paper_net.nodes[paper_id]["authors"]
    for author in authors:
        answer.append(author["org"])
    answer = "; ".join(list(set(answer)))
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-1])

{'qid': 'hard-dblp-0089', 'question': 'What institutions participated in the study of Wearable Sensor-Based Prediction Model of Timed up and Go Test in Older Adults in the DBLP citation network?', 'answer': 'Univ Nebraska Omaha, Coll Informat Sci & Technol, Omaha, NE 68182 USA; Univ Nebraska Med Ctr, Dept Biostat, Omaha, NE 68198 USA; Univ Nebraska Omaha, Dept Biomech, Omaha, NE 68182 USA'}


In [38]:
from collections import Counter
# What keywords does Blanca Rodriguez focus on most in the DBLP citation network?
for i in range(num_question_per_template):
    answer = ""
    while answer == "":
        # randomly choose a paper
        author_id = random.choice(list(id2author_dict.keys()))
        name = id2author_dict[author_id]
        question = "What keywords does {} focus on most in the DBLP citation network?".format(name)
        keywords = []
        for neighbor_id in author_net.neighbors(author_id):
            for paper in author_net[author_id][neighbor_id]["papers"]:
                keywords.extend(paper_net.nodes[paper]["keywords"])
        keywords = ', '.join(list(set(keywords)))
        answer = keywords
    questions.append({"qid": "hard-dblp-{:0>4d}".format(question_id), "question": question, "answer": answer})
    question_id += 1
print(questions[-10:])

[{'qid': 'hard-dblp-0090', 'question': 'What keywords does Shubo Liu focus on most in the DBLP citation network?', 'answer': 'Image-to-image translation,  Discriminator,  Generative adversarial networks,  Attribute label,  Multi-domain,  Unpaired dataset,  Unsupervised'}, {'qid': 'hard-dblp-0091', 'question': 'What keywords does Istvan Haller focus on most in the DBLP citation network?', 'answer': 'security-in-depth, secret-free, speculative vulnerabilities, hypervisor security'}, {'qid': 'hard-dblp-0092', 'question': 'What keywords does S. A. Abramov focus on most in the DBLP citation network?', 'answer': '39A06, 68W30, 12H10'}, {'qid': 'hard-dblp-0093', 'question': 'What keywords does Lorenz Räber focus on most in the DBLP citation network?', 'answer': 'Vessel, IVUS, Gaussian process, Lumen, Segmentation, Deep learning'}, {'qid': 'hard-dblp-0094', 'question': 'What keywords does Suvi K. Holm focus on most in the DBLP citation network?', 'answer': 'Games, Focusing, Conferences'}, {'qi

In [39]:
with jsonlines.open('/<YOUR_OWN_PATH>/ToolQA/data/questions/hard/dblp-hard.jsonl', 'w') as writer:
    for row in questions:
        writer.write(row)